"""
*****************************************************************************
* MLops                                                                     *
* OBJETIVO: Preprocesamiento del modelo vinos                               * 
* INPUT: se crea la tata insumo para prediccion.csv                         *
* OUTPUT: Predicciones                                                      *
* FECHA CREACION: 05/12/2024                                                *
* ***************************************************************************
* MODIFICACIONES
* NOMBRE WUILSON ESTACIO                    
* FECHA 05/12/2024
* VERSION 02
* DESCRIPCION Realiza la Prediccion del modelo y empaque 
******************************************************************************
"""

In [35]:
# pip install azureml-sdk

Note: you may need to restart the kernel to use updated packages.


In [36]:
from azureml.core import Workspace
from mlflow.tracking import MlflowClient

# Conecta al workspace (ajusta los parámetros si usas un archivo config.json)
workspace = Workspace.from_config()

In [63]:
# Instalar librerias
import subprocess

# Lista de librerías a instalar
librerias = ['pyspark','findspark','openpyxl','plotly','yellowbrick','jedi','seaborn','azureml-sdk']

# Iterar sobre la lista de librerías y ejecutar el comando de instalación para cada una
for libreria in librerias:
    subprocess.run(['pip', 'install', libreria])

In [38]:
%config IPCompleter.greedy=True

In [39]:
import seaborn as sns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from azureml.core.model import Model
import sys
import mlflow
import mlflow.sklearn
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, expr, asc, count, countDistinct, max, collect_list, sum, isnan, isnull, last
from pyspark.sql.functions import col as col_null
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType
# Importar bibliotecas para normalización 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
# Importar bibliotecas para Modelos
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.linalg import Vectors
from mlflow.tracking import MlflowClient

In [40]:
# Importar bibliotecas visialización
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import matplotlib

In [41]:
# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

24/12/06 21:00:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Cargamos el mejor modelos

In [42]:
from azureml.core import Workspace, Model

# Conectar al workspace
workspace_name = "Dbricksspace-war-123"  # Cambia al nombre de tu workspace
subscription_id = "e2fce082-d0f5-42e6-892d-078f098b0d47"
resource_group = "War-2025"

ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
)

# Cargar el modelo desde el registro
model_name = "RandomForest_Model"
model_version = 8
model = Model(ws, name=model_name, version=model_version)

# Descargar los artefactos del modelo
model.download(target_dir="./model", exist_ok=True)
print(f"Modelo {model_name} descargado a './model'")

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1509, in failsafe_deserialize
    return self(target_obj, data, content_type=content_type)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1375, in __call__
    data = self._unpack_content(response_data, content_type)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1543, in _unpack_content
    raise ValueError("This pipeline didn't have the RawDeserializer policy; can't deserialize")
ValueError: This pipeline didn't have the RawDeserializer policy; can't deserialize
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/msrest/serialization.py", line 1509, in failsafe_deserialize
    return self(target_obj, data, content_type=content_type)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/pytho

Modelo RandomForest_Model descargado a './model'


In [43]:
# from azureml.core import Workspace, Model

# # Conectar al workspace
# workspace_name = "Dbricksspace-war-123"  # Cambia al nombre de tu workspace
# subscription_id = "e2fce082-d0f5-42e6-892d-078f098b0d47"
# resource_group = "War-2025"

# ws = Workspace.get(
#     name=workspace_name,
#     subscription_id=subscription_id,
#     resource_group=resource_group,
# )

# # Cargar el modelo desde el registro
# model_name = "LogisticRegression_Model"
# model_version = 7
# model = Model(ws, name=model_name, version=model_version)

# # Descargar los artefactos del modelo
# model.download(target_dir="./model", exist_ok=True)
# print(f"Modelo {model_name} descargado a './model'")

In [44]:
# # Cargar el modelo con MLflow (como un modelo de pyfunc)
# import mlflow.pyfunc

# # Asumiendo que el modelo fue guardado con MLflow como 'pyfunc'
# model_path = './model/LogisticRegression_Model'  # Ruta a la carpeta donde descargaste el modelo
# model_loaded = mlflow.pyfunc.load_model(model_path)

In [45]:
ls ./model

LogisticRegression_Model/  RandomForest_Model/


In [46]:
# Verificar archivos en la carpeta del modelo
model_dir = "./model"
print("Archivos en el directorio:", os.listdir(model_dir))

Archivos en el directorio: ['LogisticRegression_Model', 'RandomForest_Model']


In [47]:
# Ruta del directorio del modelo descargado
model_subdir = "./model/RandomForest_Model"

# Verificar archivos dentro del subdirectorio del modelo
model_contents = os.listdir(model_subdir)
model_contents

['conda.yaml',
 'MLmodel',
 'python_env.yaml',
 'RandomForest_Model',
 'requirements.txt',
 'sparkml']

In [48]:
# # Ruta del directorio del modelo descargado
# model_subdir = "./model/LogisticRegression_Model"

# # Verificar archivos dentro del subdirectorio del modelo
# model_contents = os.listdir(model_subdir)
# model_contents

In [49]:
# # Verificar archivos en la carpeta principal del modelo
# model_dir = "./model/LogisticRegression_Model"
# print("Contenido en ./model:", os.listdir(model_dir))

* data: Este generalmente contiene los artefactos del modelo, como los pesos o estructuras específicas (por ejemplo, un archivo model.pkl).
* metadata: Información adicional sobre el modelo (puede incluir la definición del modelo o detalles del entrenamiento).
* treesMetadata: Específico para modelos basados en árboles, como RandomForest.

In [50]:
# Ruta del subdirectorio del modelo
# model_subdir = "./model/RandomForest_Model"
model_subdir = "./model/RandomForest_Model"

# Verificar archivos dentro del subdirectorio
print("Archivos en RandomForest_Model:", os.listdir(model_subdir))

Archivos en RandomForest_Model: ['conda.yaml', 'MLmodel', 'python_env.yaml', 'RandomForest_Model', 'requirements.txt', 'sparkml']


In [51]:
# Revisar el contenido del directorio `data`
data_dir = "./model/RandomForest_Model"
print("Archivos en el directorio 'data':", os.listdir(data_dir))

Archivos en el directorio 'data': ['conda.yaml', 'MLmodel', 'python_env.yaml', 'RandomForest_Model', 'requirements.txt', 'sparkml']


In [52]:
# Ruta al modelo guardado
model_path = "./model/RandomForest_Model"
# Cargar el modelo usando mlflow.spark
spark_model = mlflow.spark.load_model(model_path)

2024/12/06 21:01:02 INFO mlflow.spark: File './model/RandomForest_Model/sparkml' is already on DFS, copy is not necessary.


In [ ]:
# # Ruta del subdirectorio del modelo
# # model_subdir = "./model/RandomForest_Model"
# model_subdir = "./model/LogisticRegression_Model"

# # Verificar archivos dentro del subdirectorio
# print("Archivos en LogisticRegression_Model:", os.listdir(model_subdir))

# # Revisar el contenido del directorio `data`
# data_dir = "./model/LogisticRegression_Model"
# print("Archivos en el directorio 'data':", os.listdir(data_dir))

# # Ruta al modelo guardado
# model_path = "./model/LogisticRegression_Model"

# # Cargar el modelo usando mlflow.spark
# spark_model = mlflow.spark.load_model(model_path)

In [ ]:
# import os

# # Verificar contenido del directorio del modelo
# model_dir = "./model/LogisticRegression_Model"
# print("Archivos en el directorio del modelo:", os.listdir(model_dir))

# # Si es necesario, también revisa subdirectorios
# data_dir = os.path.join(model_dir, "data")
# if os.path.exists(data_dir):
#     print("Archivos en 'data':", os.listdir(data_dir))


In [ ]:
# # Cargar el modelo con MLflow
# model = mlflow.pyfunc.load_model(model_dir)

### Lectura data prediccion

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType

# Crear sesión de Spark
spark = SparkSession.builder.appName("Ejemplo").getOrCreate()


data = [
    [13.72, 1.43, 2.5, 16.7, 108.0, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285.0],
    [12.37, 0.94, 1.36, 10.6, 88.0, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520.0]
]
# Definir el esquema
schema = StructType([
    StructField("Alcohol", DoubleType(), True),
    StructField("Malic_acid", DoubleType(), True),
    StructField("Ash", DoubleType(), True),
    StructField("Alcalinity_of_ash", DoubleType(), True),
    StructField("Magnesium", DoubleType(), True),
    StructField("Total_phenols", DoubleType(), True),
    StructField("Flavanoids", DoubleType(), True),
    StructField("Nonflavanoid_phenols", DoubleType(), True),
    StructField("Proanthocyanins", DoubleType(), True),
    StructField("Color_intensity", DoubleType(), True),
    StructField("Hue", DoubleType(), True),
    StructField("OD315_diluted_wines", DoubleType(), True),
    StructField("Proline", DoubleType(), True)
])

# Crear el DataFrame
df = spark.createDataFrame(data, schema=schema)

# Mostrar el contenido del DataFrame
df.show()


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+-------------------+-------+
|Alcohol|Malic_acid| Ash|Alcalinity_of_ash|Magnesium|Total_phenols|Flavanoids|Nonflavanoid_phenols|Proanthocyanins|Color_intensity| Hue|OD315_diluted_wines|Proline|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+-------------------+-------+
|  13.72|      1.43| 2.5|             16.7|    108.0|          3.4|      3.67|                0.19|           2.04|            6.8|0.89|               2.87| 1285.0|
|  12.37|      0.94|1.36|             10.6|     88.0|         1.98|      0.57|                0.28|           0.42|           1.95|1.05|               1.82|  520.0|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+-------------------+-------+



In [54]:
df.dtypes

[('Alcohol', 'double'),
 ('Malic_acid', 'double'),
 ('Ash', 'double'),
 ('Alcalinity_of_ash', 'double'),
 ('Magnesium', 'double'),
 ('Total_phenols', 'double'),
 ('Flavanoids', 'double'),
 ('Nonflavanoid_phenols', 'double'),
 ('Proanthocyanins', 'double'),
 ('Color_intensity', 'double'),
 ('Hue', 'double'),
 ('OD315_diluted_wines', 'double'),
 ('Proline', 'double')]

## Escalamos

In [55]:
def escalar_columnas(df, columna_excluir="Target"):
    """
    Escala las columnas numéricas de un DataFrame, excluyendo la columna especificada.

    Args:
        df (pd.DataFrame): DataFrame a escalar.
        columna_excluir (str): Nombre de la columna que no se escalará.

    Returns:
        pd.DataFrame: DataFrame con las columnas seleccionadas escaladas.
    """
    # convierte a pandas
    df = df.toPandas()
    # Seleccionar las columnas a escalar
    columnas_escala = [col for col in df.columns if col != columna_excluir]
    
    # Crear un objeto StandardScaler
    scaler = StandardScaler()
    
    # Escalar las columnas seleccionadas
    df[columnas_escala] = scaler.fit_transform(df[columnas_escala])
        # convertimos a spark
    df = spark.createDataFrame(df)
    print("Columnas escaladas excluyendo:", columna_excluir)
    return df

In [56]:
df_escalado = escalar_columnas(df, columna_excluir="Target")

Columnas escaladas excluyendo: Target


In [57]:
df_escalado.show()

+-------+-------------------+-------------------+-------------------+---------+-------------+-------------------+--------------------+---------------+---------------+-------------------+-------------------+-------+
|Alcohol|         Malic_acid|                Ash|  Alcalinity_of_ash|Magnesium|Total_phenols|         Flavanoids|Nonflavanoid_phenols|Proanthocyanins|Color_intensity|                Hue|OD315_diluted_wines|Proline|
+-------+-------------------+-------------------+-------------------+---------+-------------+-------------------+--------------------+---------------+---------------+-------------------+-------------------+-------+
|    1.0| 0.9999999999999996| 0.9999999999999998| 1.0000000000000002|      1.0|          1.0| 0.9999999999999999|                -1.0|            1.0|            1.0|-0.9999999999999993| 0.9999999999999998|    1.0|
|   -1.0|-1.0000000000000004|-1.0000000000000002|-0.9999999999999997|     -1.0|         -1.0|-1.0000000000000002|                 1.0|      

In [58]:
# Eliminar la columna 'Ash' porque aporta menos de 1% a la implementacion de la variable Target
df_escalado = df_escalado.drop('Ash')

### Vectorizamos

In [59]:
from pyspark.ml.feature import VectorAssembler

# if "features" in df_escalado.columns:
#     df_escalado = df_escalado.drop("features")

assembler = VectorAssembler(inputCols=['Alcohol', 'Malic_acid','Alcalinity_of_ash', 'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins',
 'Color_intensity', 'Hue', 'OD315_diluted_wines', 'Proline'], outputCol="features")
df_escalado = assembler.transform(df_escalado)
df_escalado.show(5)

+-------+-------------------+-------------------+---------+-------------+-------------------+--------------------+---------------+---------------+-------------------+-------------------+-------+--------------------+
|Alcohol|         Malic_acid|  Alcalinity_of_ash|Magnesium|Total_phenols|         Flavanoids|Nonflavanoid_phenols|Proanthocyanins|Color_intensity|                Hue|OD315_diluted_wines|Proline|            features|
+-------+-------------------+-------------------+---------+-------------+-------------------+--------------------+---------------+---------------+-------------------+-------------------+-------+--------------------+
|    1.0| 0.9999999999999996| 1.0000000000000002|      1.0|          1.0| 0.9999999999999999|                -1.0|            1.0|            1.0|-0.9999999999999993| 0.9999999999999998|    1.0|[1.0,0.9999999999...|
|   -1.0|-1.0000000000000004|-0.9999999999999997|     -1.0|         -1.0|-1.0000000000000002|                 1.0|           -1.0|      

In [60]:
df_escalado=df_escalado.select('features')
df_escalado.show()

+--------------------+
|            features|
+--------------------+
|[1.0,0.9999999999...|
|[-1.0,-1.00000000...|
+--------------------+



In [61]:
model

Model(workspace=Workspace.create(name='Dbricksspace-war-123', subscription_id='e2fce082-d0f5-42e6-892d-078f098b0d47', resource_group='War-2025'), name=RandomForest_Model, id=RandomForest_Model:8, version=8, tags={}, properties={'flavors.spark': '{\n  "pyspark_version": "3.5.3",\n  "model_data": "sparkml",\n  "code": null,\n  "model_class": "pyspark.ml.pipeline.PipelineModel"\n}', 'flavors.python_function': '{\n  "loader_module": "mlflow.spark",\n  "python_version": "3.10.14",\n  "data": "sparkml",\n  "env": {\n    "conda": "conda.yaml",\n    "virtualenv": "python_env.yaml"\n  }\n}', 'flavors': 'spark,python_function', 'azureml.artifactPrefix': 'ExperimentRun/dcid.398ed832-d9a2-428b-9fd6-2b3108e44b61/RandomForest_Model', 'model_json': '{"run_id": "398ed832-d9a2-428b-9fd6-2b3108e44b61", "artifact_path": "RandomForest_Model", "utc_time_created": "2024-12-06 20:52:20.718155", "model_uuid": "b3a0f719fcb04e06a0fcb28a03106ee4", "flavors": {"spark": {"pyspark_version": "3.5.3", "model_data": "

### Predecimos

In [62]:
# Realizar predicciones usando el modelo cargado
from joblib import load
predicciones = spark_model.transform(df_escalado)
# Mostrar las predicciones
predicciones.show()

+--------------------+-----------------+-----------------+----------+
|            features|    rawPrediction|      probability|prediction|
+--------------------+-----------------+-----------------+----------+
|[1.0,0.9999999999...|[0.0,9.0,1.0,0.0]|[0.0,0.9,0.1,0.0]|       1.0|
|[-1.0,-1.00000000...|[0.0,0.0,5.0,5.0]|[0.0,0.0,0.5,0.5]|       2.0|
+--------------------+-----------------+-----------------+----------+



In [ ]:
print('fin prediccioens')